In [ ]:
import pandas as pd
import os 
import dask.dataframe as dd
import numpy as np
from dask.diagnostics import ProgressBar
import gc
import glob
import matplotlib.pyplot as plt
import math
import seaborn as sns
import sys
import re
import rdata
import statistics
import scipy.stats
import collections

from decimal import Decimal, ROUND_HALF_UP

from statistics import mean, median,variance,stdev

from sklearn.metrics import mean_squared_error

In [ ]:
def ExtractEnvCount_FromALTC(DATAFRAME):
    
    ALTC_List = list(DATAFRAME["ALTC"])

    EnvCount_List = []

    for ALTC in ALTC_List:
        
        EnvCount = re.findall(r"ALTC_(\w+)_\d{3}", ALTC)[0]
        
        EnvCount_List.append(EnvCount)
    
    DATAFRAME["EnvCount"] = EnvCount_List    
        
    return DATAFRAME

In [ ]:
df_TimeStep_Path =  r"df_Pridiciton_Success_Day.csv"
df = dd.read_csv(df_TimeStep_Path , assume_missing=True, dtype = 'object').compute()
df = ExtractEnvCount_FromALTC(df)
df = df[['ALTC', 'Gene','Date(NoHour)','Pridiciton_Success', 'EnvCount']]

In [ ]:
df_08 = df[df["EnvCount"] == "08"]
df_08Night = df[df["EnvCount"] == "08Night"]
df_08Day = df[df["EnvCount"] == "08Day"]

# Count the number of days of successful predictions at each ALTC and Gene
df_08_gb = df_08.groupby(["ALTC" , 'Gene' , "Pridiciton_Success"], as_index=False).count()
df_08_gb = df_08_gb[df_08_gb["Pridiciton_Success"] == "1"]
df_08_gb = df_08_gb[['ALTC', 'Gene', 'Date(NoHour)']]
df_08_gb = df_08_gb.rename(columns={'Date(NoHour)': 'Pridiction_Success_Control'})

df_08Night_gb = df_08Night.groupby(["ALTC" , 'Gene' , "Pridiciton_Success"], as_index=False).count()
df_08Night_gb = df_08Night_gb[df_08Night_gb["Pridiciton_Success"] == "1"]
df_08Night_gb = df_08Night_gb[['ALTC', 'Gene', 'Date(NoHour)']]
df_08Night_gb = df_08Night_gb.rename(columns={'Date(NoHour)': 'Pridiction_Success_Night'})

df_08Day_gb = df_08Day.groupby(["ALTC" , 'Gene' , "Pridiciton_Success"], as_index=False).count()
df_08Day_gb = df_08Day_gb[df_08Day_gb["Pridiciton_Success"] == "1"]
df_08Day_gb = df_08Day_gb[['ALTC', 'Gene', 'Date(NoHour)']]
df_08Day_gb = df_08Day_gb.rename(columns={'Date(NoHour)': 'Pridiction_Success_Day'})

def Make_LackData(ALTC_LIST , GENE_LIST , TYPE):
    
    if TYPE == "Control":
        
        Pridiction_Type = "Pridiction_Success_Control"
    elif TYPE == "Day":
        Pridiction_Type = "Pridiction_Success_Day"
    elif TYPE == "Night":
        Pridiction_Type = "Pridiction_Success_Night"
        
    
    df_LackData_List = []

    for ALTC in ALTC_LIST:
        for Gene in GENE_LIST:
            df_LackData_List.append([ALTC, Gene , "0"])

    df_LackData = pd.DataFrame(df_LackData_List , columns =['ALTC' , 'Gene' , Pridiction_Type])
    
    return df_LackData

GeneList = list(set((list(set(df_08["Gene"]))) + (list(set(df_08Night["Gene"]))) +(list(set(df_08Day["Gene"])))))

df_LackData_08 = Make_LackData(list(set(df_08["ALTC"])) , GeneList , "Control")
df_LackData_08Night = Make_LackData(list(set(df_08Night["ALTC"])) , GeneList , "Night")
df_LackData_08Day = Make_LackData(list(set(df_08Day["ALTC"])) , GeneList , "Day")

df_08_gb_Concat = pd.concat([df_08_gb , df_LackData_08])
df_08Night_gb_Concat = pd.concat([df_08Night_gb , df_LackData_08Night])
df_08Day_gb_Concat = pd.concat([df_08Day_gb , df_LackData_08Day])

df_08_gb_Duplicates = df_08_gb_Concat[~df_08_gb_Concat.duplicated(subset=['ALTC' , 'Gene'], keep=False)]
df_08Night_gb_Duplicates = df_08Night_gb_Concat[~df_08Night_gb_Concat.duplicated(subset=['ALTC' , 'Gene'], keep=False)]
df_08Day_gb_Duplicates = df_08Day_gb_Concat[~df_08Day_gb_Concat.duplicated(subset=['ALTC' , 'Gene'], keep=False)]

df_08_gb = pd.concat([df_08_gb , df_08_gb_Duplicates])
df_08Day_gb = pd.concat([df_08Day_gb , df_08Day_gb_Duplicates])
df_08Night_gb = pd.concat([df_08Night_gb , df_08Night_gb_Duplicates])

df_08_gb['ALTC_Number'] = df_08_gb['ALTC'].str[-3:]
df_08Day_gb['ALTC_Number'] = df_08Day_gb['ALTC'].str[-3:]
df_08Night_gb['ALTC_Number'] = df_08Night_gb['ALTC'].str[-3:]

df_08DayCon_gb = pd.merge(df_08_gb , df_08Day_gb , how = "inner" , on = ["ALTC_Number" , "Gene"])
df_08NightCon_gb = pd.merge(df_08_gb , df_08Night_gb , how = "inner" , on = ["ALTC_Number" , "Gene"])
df_DayNight_gb = pd.merge(df_08Day_gb , df_08Night_gb , how = "inner" , on = ["ALTC_Number" , "Gene"])

df_08DayCon_gb = df_08DayCon_gb.astype({'Pridiction_Success_Control': int , "Pridiction_Success_Day" : int})
df_08NightCon_gb = df_08NightCon_gb.astype({'Pridiction_Success_Control': int , "Pridiction_Success_Night" : int})
df_DayNight_gb = df_DayNight_gb.astype({'Pridiction_Success_Night': int , "Pridiction_Success_Day" : int})

In [ ]:
# Con-Dayのscript
df_08DayCon_gb = df_08DayCon_gb.rename(columns={'ALTC_x': 'ALTC_Control','ALTC_y': 'ALTC_Day'})

df_08Day0Con9 = df_08DayCon_gb[(df_08DayCon_gb["Pridiction_Success_Control"] == 9) & (df_08DayCon_gb["Pridiction_Success_Day"] == 0)]
df_08Day0Con9 = df_08Day0Con9[['ALTC_Control', 'Gene', 'ALTC_Day']]
df_08Day0Con9["Type"] = "Day0Con9"

df_08Day9Con0 = df_08DayCon_gb[(df_08DayCon_gb["Pridiction_Success_Control"] == 0) & (df_08DayCon_gb["Pridiction_Success_Day"] == 9)]
df_08Day9Con0 = df_08Day9Con0[['ALTC_Control', 'Gene', 'ALTC_Day']]
df_08Day9Con0["Type"] = "Day9Con0"


# Con-Nightのscript
df_08NightCon_gb = df_08NightCon_gb.rename(columns={'ALTC_x': 'ALTC_Control','ALTC_y': 'ALTC_Night'})

df_08Night0Con9 = df_08NightCon_gb[(df_08NightCon_gb["Pridiction_Success_Control"] == 9) & (df_08NightCon_gb["Pridiction_Success_Night"] == 0)]
df_08Night0Con9 = df_08Night0Con9[['ALTC_Control', 'Gene', 'ALTC_Night']]
df_08Night0Con9["Type"] = "Night0Con9"

df_08Night9Con0 = df_08NightCon_gb[(df_08NightCon_gb["Pridiction_Success_Control"] == 0) & (df_08NightCon_gb["Pridiction_Success_Night"] == 9)]
df_08Night9Con0 = df_08Night9Con0[['ALTC_Control', 'Gene', 'ALTC_Night']]
df_08Night9Con0["Type"] = "Night9Con0"


# DayNigthのスクリプト
df_DayNight_gb = df_DayNight_gb.rename(columns={'ALTC_x': 'ALTC_Day','ALTC_y': 'ALTC_Night'})

df_Day0Night9_gb = df_DayNight_gb[(df_DayNight_gb["Pridiction_Success_Day"] == 0) & (df_DayNight_gb["Pridiction_Success_Night"] == 9)]
df_Day0Night9_gb = df_Day0Night9_gb[['ALTC_Day', 'Gene', 'ALTC_Night']]
df_Day0Night9_gb["Type"] = "Day0Night9"

df_Day9Night0_gb= df_DayNight_gb[(df_DayNight_gb["Pridiction_Success_Day"] == 9) & (df_08NightCon_gb["Pridiction_Success_Night"] == 0)]
df_Day9Night0_gb = df_Day9Night0_gb[['ALTC_Day', 'Gene', 'ALTC_Night']]
df_Day9Night0_gb["Type"] = "Day9Night0"

In [ ]:
df_Day = pd.concat([df_08Day0Con9 , df_08Day9Con0])
df_Night = pd.concat([df_08Night0Con9 , df_08Night9Con0])
df_DayNight = pd.concat([df_Day0Night9_gb , df_Day9Night0_gb])

df_Day = df_Day.rename({'ALTC_Day': 'ALTC_Experiment'}, axis='columns')
df_Night = df_Night.rename({'ALTC_Night': 'ALTC_Experiment'}, axis='columns')

df_Control_Experiment = pd.concat([df_Day , df_Night])

In [ ]:
df_Control_Experiment.to_csv(r"df_DayNight0Con9.csv" , index=False)
df_DayNight.to_csv(r"df_DayNight09.csv" , index=False)

In [ ]:
df_All_Path = r"df_all_fit_filterd.csv"

usecols = ['FIT_Test_Value', 'ALTC', 'EnvCount','Gene','Temperature', 'Radiation', 'FIT_Reference_Value', 'Date']

df = dd.read_csv(df_All_Path,assume_missing=True, usecols=usecols,dtype={'FIT_Test_Value': float,'ALTC': str,'EnvCount': str,
       'Gene': str, "Temperature":float , "Radiation":float ,"FIT_Correct_Value": float}).compute()

df['Date(NoHour)'] = pd.to_datetime(df['Date']).dt.date

In [ ]:
df_DayNight = df_DayNight.rename(columns={'ALTC_Day'  : 'ALTC_Control'  , "ALTC_Night" : "ALTC_Experiment"})
df_0990 = pd.concat([df_Control_Experiment , df_DayNight])

In [ ]:
df_all_list = []

for DataType in (['Day0Con9' , 'Day9Con0', 'Night0Con9', 'Night9Con0' , "Day0Night9" , "Day9Night0"]):
    
    df_0990_2Dlist = (df_0990[df_0990["Type"] == DataType]).to_numpy().tolist()

    for SingleList in df_0990_2Dlist:
        
        ALTC_Control = SingleList[0]
        ALTC_Experiment = SingleList[2]
        Gene = SingleList[1]
        DataType = SingleList[3]
        
        print(ALTC_Control, ALTC_Experiment, Gene)
        
        df_tmp1 = df[ (df["ALTC"] == ALTC_Control) & (df["Gene"] == Gene) ]
        df_tmp1 = df_tmp1[["ALTC" , "Gene" , "FIT_Test_Value" , "Date" , "Date(NoHour)"]]
        df_tmp1 = df_tmp1.rename(columns={'FIT_Test_Value'  : 'FIT_Control_Value', "ALTC":"ALTC_Control"})
        
        df_tmp2 = df[ (df["ALTC"] == ALTC_Experiment) & (df["Gene"] == Gene) ]
        df_tmp2 = df_tmp2[["ALTC" , "Gene" , "FIT_Test_Value" , "Date" , "Date(NoHour)"]]
        df_tmp2 = df_tmp2.rename(columns={'FIT_Test_Value'  : 'FIT_Experiment_Value'  , "ALTC":"ALTC_Experiment"})
        
        if (len(df_tmp1) != 108) or (len(df_tmp2) != 108):
            pass
        else:
            df_tmp = pd.merge(df_tmp1 , df_tmp2 , on = ["Date" , "Date(NoHour)" , "Gene"])
            df_tmp["Type"] = DataType
            df_all_list.append(df_tmp)

df_all = pd.concat(df_all_list)            

In [ ]:
df_all.to_csv(r"DayNightOnlydataset_0990Days.csv" , index = False)
df_all = pd.read_csv(r"DayNightOnlydataset_0990Days.csv")

In [ ]:
df_all["ALTC"] = df_all['ALTC_Control'].str[-3:]

In [ ]:
Corr_df = df_all.groupby(['ALTC', 'Gene' , "Date(NoHour)" , "Type"]).apply(lambda x: x['FIT_Control_Value'].corr(x['FIT_Experiment_Value'])).reset_index(name='Corr')

def calculate_rmse(group):
    actual = group['FIT_Control_Value']
    predicted = group['FIT_Experiment_Value']
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    return rmse

RMSE_df = df_all.groupby(['ALTC', 'Gene' , "Date(NoHour)", "Type"]).apply(calculate_rmse).reset_index(name='RMSE')

In [ ]:
df_Corr_RMSE = pd.merge(Corr_df,RMSE_df,how="inner",on = ['ALTC', 'Gene' , "Date(NoHour)" , "Type"])

In [ ]:
df_Corr_RMSE['Corr_Success'] = df_Corr_RMSE['Corr'].apply(lambda x: 1 if x > 0.8 else 0)

df_Corr_RMSE['RMSE_Success'] = df_Corr_RMSE['RMSE'].apply(lambda x: 1 if x < 0.4 else 0)

df_Corr_RMSE['Pridiciton_Success'] = ((df_Corr_RMSE['Corr'] > 0.8) & (df_Corr_RMSE['RMSE'] < 0.4)).astype(int)

In [ ]:
df_Corr_RMSE.to_csv(r"DayNightOnlydataset_0990Days_PrdSucDay.csv", index = False)